In [ ]:
from pandas import DataFrame,read_csv,read_excel,merge,melt,pivot_table
import numpy
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import sys
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from factor_analyzer.factor_analyzer import calculate_kmo

## 데이터 준비

In [ ]:
origin=read_excel('행정동병합테스트22.xlsx')
origin.head()

In [ ]:
origin.columns

## 결측치는 0으로 처리

> 실제 조사된 값이므로 없다면 관측되지 아니함으로 판단

In [ ]:
origin.isna().sum()
origin1=origin.fillna(0)

In [ ]:
origin1.columns

## 데이터 스케일링

In [ ]:
scaler=MinMaxScaler()
df = origin1.copy()
df['수변점수minmax']=scaler.fit_transform(df[['수변점수']])
df.head()

## 파생변수(`그린지수`) 생성

In [ ]:
df['그린지수'] = df['세대당공원면적minmax'] + df['수변점수minmax']
df.head()

## 파생 변수의 값 정렬

In [8]:
df['그린지수'].value_counts().sort_index()

그린지수
0.000000    8
0.000031    1
0.000042    1
0.000105    1
0.000167    2
           ..
1.272919    1
1.451516    1
1.577391    1
1.660735    1
1.885626    1
Name: count, Length: 342, dtype: int64

## 결측치 확인

In [9]:
df['그린지수'].isna().sum()

0

In [11]:
# df['그린지수'].filln(0)
# df['그린지수'].isnu().sum()

In [12]:
df['그린지수'].unique()

array([3.52897513e-01, 6.35539618e-02, 1.00000000e+00, 1.00119183e+00,
       2.25820953e-03, 2.43593899e-03, 5.06006210e-03, 1.06010392e-01,
       1.02455803e-03, 4.88651452e-02, 2.40143856e-02, 0.00000000e+00,
       3.09458343e-03, 1.46365433e-03, 6.16100198e-01, 1.00153684e+00,
       1.55356452e-02, 5.95916404e-03, 1.86093193e-03, 1.96547867e-03,
       1.30683422e-02, 5.00000000e-01, 3.45004234e-04, 1.44274498e-02,
       2.71821518e-04, 4.03209585e-01, 1.42183563e-03, 1.00016727e+00,
       9.97375877e-03, 3.03185539e-04, 9.93194007e-04, 1.16046879e-03,
       2.59275909e-03, 1.00044955e+00, 4.20696072e-02, 3.00292731e-01,
       1.00187139e+00, 1.00129638e+00, 1.00012546e+00, 1.00637735e-01,
       1.15001411e-03, 1.02882354e+00, 4.04850969e-01, 1.00131729e+00,
       1.00030319e+00, 1.00074228e+00, 6.00700463e-01, 1.00040773e+00,
       7.12660610e-01, 2.84367126e-03, 1.00785146e+00, 1.00026137e+00,
       1.06665900e+00, 1.00055410e+00, 1.00071092e+00, 1.00308413e+00,
      

In [13]:
df.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계_명', '면적', '구성비', '행정동별_공원수',
       '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소', '면수', '하천및한강여부', '아동인구',
       '노인인구', '장애인_인구', '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수',
       '행정동별버스정류장수', '수변점수minmax', '그린지수'],
      dtype='object')

## 지수 파일 생성에 사용될 컬럼 추출

In [14]:
df1=df.filter(['시군구코드','시군구명','행정동코드','행정동코드수정','읍면동명','수변점수','수변점수minmax', '세대당공원면적minmax', '그린지수'])
df1.head()

,시군구코드,시군구명,행정동코드,읍면동명,수변점수,수변점수minmax,세대당공원면적minmax,그린지수
0,11110,종로구,1111051500,청운효자동,0,0.0,0.352898,0.352898
1,11110,종로구,1111053000,사직동,0,0.0,0.063554,0.063554
2,11110,종로구,1111054000,삼청동,0,0.0,1.000000,1.000000
3,11110,종로구,1111055000,부암동,10,1.0,0.000000,1.000000
4,11110,종로구,1111056000,평창동,10,1.0,0.001192,1.001192


## 지수 파일 생성

In [15]:
df1.to_excel('환경자족성지수1.xlsx', index=False)

In [16]:
df2=read_excel('환경자족성지수1.xlsx')
df2.head()

,시군구코드,시군구명,행정동코드,읍면동명,수변점수,수변점수minmax,세대당공원면적minmax,그린지수
0,11110,종로구,1111051500,청운효자동,0,0.0,0.352898,0.352898
1,11110,종로구,1111053000,사직동,0,0.0,0.063554,0.063554
2,11110,종로구,1111054000,삼청동,0,0.0,1.000000,1.000000
3,11110,종로구,1111055000,부암동,10,1.0,0.000000,1.000000
4,11110,종로구,1111056000,평창동,10,1.0,0.001192,1.001192


## 이상치 `inf`, `-inf` 처리

In [17]:
# DataFrame(df2)에서 '그린지수' 열을 선택
accessibility_series = df2['그린지수']

# 'inf'와 '-inf'를 0으로 대체
accessibility_series = accessibility_series.replace([np.inf, -np.inf], 0)

# 변경된 '그린지수' 열을 DataFrame에 할당
df2['그린지수'] = accessibility_series

In [18]:
df2.to_excel("환경자족성지수1.xlsx", index=False)

In [19]:
df2 = read_excel("환경자족성지수1.xlsx")
df2.head()

,시군구코드,시군구명,행정동코드,읍면동명,수변점수,수변점수minmax,세대당공원면적minmax,그린지수
0,11110,종로구,1111051500,청운효자동,0,0.0,0.352898,0.352898
1,11110,종로구,1111053000,사직동,0,0.0,0.063554,0.063554
2,11110,종로구,1111054000,삼청동,0,0.0,1.000000,1.000000
3,11110,종로구,1111055000,부암동,10,1.0,0.000000,1.000000
4,11110,종로구,1111056000,평창동,10,1.0,0.001192,1.001192


## 지수 정규화

In [20]:
df2['그린지수minmax'] = scaler.fit_transform(df2[['그린지수']])
df2

,시군구코드,시군구명,행정동코드,읍면동명,수변점수,수변점수minmax,세대당공원면적minmax,그린지수,그린지수minmax
0,11110,종로구,1111051500,청운효자동,0,0.0,0.352898,0.352898,0.187151
1,11110,종로구,1111053000,사직동,0,0.0,0.063554,0.063554,0.033704
2,11110,종로구,1111054000,삼청동,0,0.0,1.000000,1.000000,0.530328
3,11110,종로구,1111055000,부암동,10,1.0,0.000000,1.000000,0.530328
4,11110,종로구,1111056000,평창동,10,1.0,0.001192,1.001192,0.530960
...,...,...,...,...,...,...,...,...,...
421,11740,강동구,1174065000,성내제2동,4,0.4,0.000188,0.400188,0.212231
422,11740,강동구,1174066000,성내제3동,9,0.9,0.000366,0.900366,0.477489
423,11740,강동구,1174068500,길동,0,0.0,0.004297,0.004297,0.002279
424,11740,강동구,1174069000,둔촌제1동,7,0.7,0.000000,0.700000,0.371230


## 최종 데이터 저장

In [ ]:
df2.to_excel('환경자족성지수1.xlsx',index=False)